In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pywt
import matlab.engine
import dippykit as dip
import ssim.ssimlib as pyssim
from skimage.exposure import match_histograms 
from PIL import Image
from keras.models import load_model
eng = matlab.engine.start_matlab()
eng.addpath("Function\InputWeights")
eng.addpath("Function\InputWeights_U")
eng.addpath("Function")

[2023-11-30 13:48:43,762] DEBUG: Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
[2023-11-30 13:48:44,130] DEBUG: Creating converter from 7 to 5
[2023-11-30 13:48:44,131] DEBUG: Creating converter from 5 to 7
[2023-11-30 13:48:44,131] DEBUG: Creating converter from 7 to 5
[2023-11-30 13:48:44,133] DEBUG: Creating converter from 5 to 7
[2023-11-30 13:48:46,929] WARNING: From c:\Users\ajayr\miniconda3\envs\dip_project\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



'C:\\Users\\ajayr\\Documents\\Masters\\3rd Sem\\Digital Image Processing\\Project\\Function\\InputWeights_U;C:\\Users\\ajayr\\Documents\\Masters\\3rd Sem\\Digital Image Processing\\Project\\Function\\InputWeights;C:\\Users\\ajayr\\Documents\\MATLAB;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addon_enable_disable_management\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addon_updates\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons\\cef;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons\\fileexchange;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons\\supportpackages;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_common\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_install_location\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_product;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_product_support\\matlab;C:\\P

## Importing Dataset

In [2]:
def read_files(file_pth,base):
  imgs = []
  for files in file_pth:
    img=cv2.imread(base+files)
    img=cv2.resize(img,(512,512))
    # mean = np.mean(img
    # std_dev = np.std(img)
    # img = (img - mean) / std_dev
    # img=(img-np.min(img))/(np.max(img)-np.min(img))
    imgs.append(img)
  return imgs

In [3]:
base_pth = 'CURE-TSD/'
# noisy_pths = sorted(os.listdir(base_pth+'Noisy_images/'))
# gt_pths = sorted(os.listdir(base_pth+'Ground_truths/'))
gt_pths = [file for file in sorted(os.listdir(base_pth+'Ground_truths/')) if not file.startswith("._")]
noisy_pths = [file for file in sorted(os.listdir(base_pth+'Noisy_images/')) if not file.startswith("._")]


print(len(noisy_pths))


1150


In [4]:
name_dict = {
    '01_01.mp4.png':'deco_lvl_1',
    '01_02.mp4.png':'deco_lvl_2',
    '01_03.mp4.png':'deco_lvl_3',
    '01_04.mp4.png':'deco_lvl_4',
    '01_05.mp4.png':'deco_lvl_5',
    '02_01.mp4.png':'lens_blur_lvl_1' ,
    '02_02.mp4.png':'lens_blur_lvl_2' ,
    '02_03.mp4.png':'lens_blur_lvl_3' ,
    '02_04.mp4.png':'lens_blur_lvl_4' ,
    '02_05.mp4.png':'lens_blur_lvl_5' ,
    '03_01.mp4.png':'codex_lvl_1' ,
    '03_02.mp4.png':'codex_lvl_2' ,
    '03_03.mp4.png':'codex_lvl_3' ,
    '03_04.mp4.png':'codex_lvl_4' ,
    '03_05.mp4.png':'codex_lvl_5' ,
    '04_01.mp4.png':'dark_lvl_1' ,
    '04_02.mp4.png':'dark_lvl_2' ,
    '04_03.mp4.png':'dark_lvl_3' ,
    '04_04.mp4.png':'dark_lvl_4' ,
    '04_05.mp4.png':'dark_lvl_5' ,
    '05_01.mp4.png':'dirty_lvl_1' ,
    '05_02.mp4.png':'dirty_lvl_2' ,
    '05_03.mp4.png':'dirty_lvl_3' ,
    '05_04.mp4.png':'dirty_lvl_4' ,
    '05_05.mp4.png':'dirty_lvl_5' ,
    '06_01.mp4.png':'exp_lvl_1',
    '06_02.mp4.png':'exp_lvl_2',
    '06_03.mp4.png':'exp_lvl_3',
    '06_04.mp4.png':'exp_lvl_4',
    '06_05.mp4.png':'exp_lvl_5',
    '07_01.mp4.png':'gauss_blur_lvl_1' ,
    '07_02.mp4.png':'gauss_blur_lvl_2' ,
    '07_03.mp4.png':'gauss_blur_lvl_3' ,
    '07_04.mp4.png':'gauss_blur_lvl_4' ,
    '07_05.mp4.png':'gauss_blur_lvl_5' ,
    '08_01.mp4.png':'noise_lvl_1' ,
    '08_02.mp4.png':'noise_lvl_2' ,
    '08_03.mp4.png':'noise_lvl_3' ,
    '08_04.mp4.png':'noise_lvl_4' ,
    '08_05.mp4.png':'noise_lvl_5' ,
    '09_01.mp4.png':'rain_lvl_1' ,
    '09_02.mp4.png':'rain_lvl_2' ,
    '09_03.mp4.png':'rain_lvl_3' ,
    '09_04.mp4.png':'rain_lvl_4' ,
    '09_05.mp4.png':'rain_lvl_5' ,
    '10_01.mp4.png':'shadow_lvl_1',
    '10_02.mp4.png':'shadow_lvl_2',
    '10_03.mp4.png':'shadow_lvl_3',
    '10_04.mp4.png':'shadow_lvl_4',  
    '10_05.mp4.png':'shadow_lvl_5',
    '11_01.mp4.png':'snow_lvl_1' ,
    '11_02.mp4.png':'snow_lvl_2' ,
    '11_03.mp4.png':'snow_lvl_3' ,
    '11_04.mp4.png':'snow_lvl_4' ,
    '11_05.mp4.png':'snow_lvl_5' ,
    '12_01.mp4.png':'haze_lvl_1' ,
    '12_02.mp4.png':'haze_lvl_2' ,
    '12_03.mp4.png':'haze_lvl_3' ,
    '12_04.mp4.png':'haze_lvl_4' ,
    '12_05.mp4.png':'haze_lvl_5' ,

}
category_lists = {key: [] for key in name_dict.values()}
for i in noisy_pths:
    file_split = i.split('_')
    keyword = str(file_split[-2])+'_'+str(file_split[-1])

    # Check if the keyword is in the name_dict
    if keyword in name_dict:
        # Append to the corresponding list based on the value in name_dict
        category_list = name_dict[keyword]
        category_lists[category_list].append(i)

# Print the results
for category, file_list in category_lists.items():
    print(f"{category}: {file_list}")

deco_lvl_1: ['01_04_01_01_01.mp4.png', '01_06_01_01_01.mp4.png', '01_19_01_01_01.mp4.png', '01_22_01_01_01.mp4.png', '01_23_01_01_01.mp4.png', '01_29_01_01_01.mp4.png', '01_34_01_01_01.mp4.png', '01_38_01_01_01.mp4.png', '01_42_01_01_01.mp4.png', '01_48_01_01_01.mp4.png', '02_01_01_01_01.mp4.png', '02_08_01_01_01.mp4.png', '02_12_01_01_01.mp4.png', '02_20_01_01_01.mp4.png', '02_22_01_01_01.mp4.png', '02_24_01_01_01.mp4.png', '02_30_01_01_01.mp4.png', '02_32_01_01_01.mp4.png', '02_47_01_01_01.mp4.png', '02_49_01_01_01.mp4.png']
deco_lvl_2: ['01_04_01_01_02.mp4.png', '01_06_01_01_02.mp4.png', '01_19_01_01_02.mp4.png', '01_22_01_01_02.mp4.png', '01_23_01_01_02.mp4.png', '01_29_01_01_02.mp4.png', '01_34_01_01_02.mp4.png', '01_38_01_01_02.mp4.png', '01_42_01_01_02.mp4.png', '01_48_01_01_02.mp4.png', '02_01_01_01_02.mp4.png', '02_08_01_01_02.mp4.png', '02_12_01_01_02.mp4.png', '02_20_01_01_02.mp4.png', '02_22_01_01_02.mp4.png', '02_24_01_01_02.mp4.png', '02_30_01_01_02.mp4.png', '02_32_01_01

In [5]:
small_noisy_lst = {}
for name, cat in category_lists.items():
    small_noisy_lst[name] = cat[0:10]


In [6]:
noisy_imgs = {} 
for i,j in small_noisy_lst.items():
    noisy_files = base_pth+'Noisy_images/'
    noisy_imgs[i] = read_files(j,noisy_files)

gt_files = base_pth+'Ground_truths/'
gt_imgs= read_files(gt_pths[0:10],gt_files)

## Functions

In [7]:
def display_plots(gt_img,noisy_img,denoisy_img):
    # Create a figure with three subplots
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))

    # Display the first image
    ax1.imshow(gt_img, cmap='gray')
    ax1.set_title('Ground Truth')
    ax1.axis('off')

    # Display the second image
    ax2.imshow(noisy_img, cmap='gray')
    ax2.set_title('Noisy Image')
    ax2.axis('off')

    # Display the third image
    ax3.imshow(denoisy_img, cmap='gray')
    ax3.set_title('Denoised Image')
    ax3.axis('off')

    # Adjust layout to prevent clipping of titles
    plt.tight_layout()

    # Show the plot
    plt.show()


In [8]:
def nlmeans_denoise(image, h=3, templateWindowSize=7, searchWindowSize=21):
    # Perform Non-Local Means Denoising using OpenCV
    return cv2.fastNlMeansDenoising(image, None, h, templateWindowSize, searchWindowSize)

def wavelet_denoise(img, threshold=20):

    channels = cv2.split(img)

    # Apply wavelet denoising to each channel independently
    denoised_channels = []
    for channel in channels:
        # Perform 2D wavelet transform
        coeffs = pywt.dwt2(channel, 'bior1.3')

        # Thresholding: Set small coefficients to zero
        coeffs = tuple(map(lambda x: pywt.threshold(x, threshold, mode='soft'), coeffs))

        # Inverse 2D wavelet transform
        denoised_channel = pywt.idwt2(coeffs, 'bior1.3')

        # Clip values to be in the valid range [0, 255]
        denoised_channel = np.clip(denoised_channel, 0, 255)

        # Convert to uint8 (required for image display)
        denoised_channel = denoised_channel.astype(np.uint8)

        denoised_channels.append(denoised_channel)

    # Merge the denoised channels back into an RGB image
    denoised_img = cv2.merge(denoised_channels)

    return denoised_img

def combine_denoising_methods(image):

    # Apply NLMeans denoising
    nlmeans_denoised = nlmeans_denoise(image)

    # Apply wavelet denoising
    wavelet_denoised = wavelet_denoise(image)

    nlmeans_denoised = np.array(nlmeans_denoised)
    wavelet_denoised = np.array(wavelet_denoised)

    nlmeans_denoised_norm = nlmeans_denoised / 255.0
    wavelet_denoised_norm = cv2.resize(wavelet_denoised, (nlmeans_denoised.shape[1], nlmeans_denoised.shape[0])) / 255.0


    # Combine the results (you can experiment with different blending strategies)
    combined_denoised_norm = 0.2 * nlmeans_denoised_norm + 0.8 * wavelet_denoised_norm

    combined_denoised = (combined_denoised_norm * 255).astype(np.uint8)

    return combined_denoised

def convert(img):
    image= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    return image

In [9]:
def psnr_2_lsts(lst1,gt_lst2):
    psnrs = []
    for i,j in zip(lst1,gt_lst2):
        val = cv2.PSNR(j, i)
        psnrs.append(val)
    return psnrs

def plot_normal_distribution(old_psnrs, new_psnrs):
    # Calculate mean and standard deviation for each dataset
    mean1 = np.mean(old_psnrs)
    std_dev1 = np.std(old_psnrs)

    mean2 = np.mean(new_psnrs)
    std_dev2 = np.std(new_psnrs)

    # Create a range of x values
    x = np.linspace(min(old_psnrs + new_psnrs), max(old_psnrs + new_psnrs), 100)

    # Generate probability density functions (PDFs) for each dataset
    pdf1 = 1 / (std_dev1 * np.sqrt(2 * np.pi)) * np.exp(-(x - mean1)**2 / (2 * std_dev1**2))
    pdf2 = 1 / (std_dev2 * np.sqrt(2 * np.pi)) * np.exp(-(x - mean2)**2 / (2 * std_dev2**2))

    # Plot the normal distribution curves
    plt.plot(x, pdf1, label='old_psnrs', color='blue')
    plt.plot(x, pdf2, label='new_psnrs', color='red')

    plt.xlabel('X-axis')
    plt.ylabel('Probability Density')
    plt.title('Normal Distribution of Data')
    plt.legend()

    plt.show()

In [10]:
class Evaluation_Metrics:
    def __init__(self):
        pass  

    def psnr(self, img1, img2):
        return dip.metrics.PSNR(img1, img2)

    def mse(self, img1, img2):
        return dip.metrics.MSE(img1, img2)

    def ssim(self, img1, img2):
        img1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
        img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
        return dip.metrics.SSIM(img1, img2)[0]
    def cwssim(self,img1,img2):
        img1_pillow = Image.fromarray(img1)
        img2_pillow = Image.fromarray(img2)
        similarity = pyssim.SSIM(img1_pillow).cw_ssim_value(img2_pillow)
        return similarity
    def unique(self, img1, img2):
        return eng.mslUNIQUE(img1, img2)

    def ms_unique(self, img1, img2):
        return eng.mslMSUNIQUE(img1, img2)

    def summer(self, img1, img2):
        return eng.SUMMER(img1, img2)

    def csv(self, img1, img2):
        return eng.csv(img1, img2)
metrics=Evaluation_Metrics()

In [11]:
def evaluation_metrics(denoisy_img,noisy_img,gt):
    eval_dict = {}

    # ******************* PSNR *******************
    psnr_noisy = metrics.psnr(noisy_img,gt)
    psnr_denoisy =metrics.psnr(denoisy_img,gt)
    eval_dict['PSNR_noisy'] = psnr_noisy
    eval_dict['PSNR_denoisy'] = psnr_denoisy

    # ******************* SSIM ******************* 
    ssim_noisy = metrics.ssim(noisy_img,gt)
    ssim_denoisy =metrics.ssim(denoisy_img,gt)
    eval_dict['SSIM_noisy'] = ssim_noisy
    eval_dict['SSIM_denoisy'] = ssim_denoisy

    # ******************* CW-SSIM ******************* 
    cw_ssim_noisy = metrics.cwssim(noisy_img,gt)
    cw_ssim_denoisy = metrics.cwssim(denoisy_img,gt)
    eval_dict['CW-SSIM_noisy'] = cw_ssim_noisy
    eval_dict['CW-SSIM_denoisy'] = ssim_denoisy
    # ******************* UNIQUE ******************* 
    unique_noisy = metrics.unique(noisy_img,gt)
    unique_denoisy =metrics.unique(denoisy_img,gt)
    eval_dict['unique_noisy'] = unique_noisy
    eval_dict['unique_denoisy'] = unique_denoisy

    # ******************* MS-UNIQUE ******************* 
    ms_unique_noisy = metrics.ms_unique(noisy_img,gt)
    ms_unique_denoisy =metrics.ms_unique(denoisy_img,gt)
    eval_dict['ms_unique_noisy'] = ms_unique_noisy
    eval_dict['ms_unique_denoisy'] = ms_unique_denoisy

    # ******************* CSV ******************* 
    csv_noisy = metrics.csv(noisy_img,gt)
    csv_denoisy =metrics.csv(denoisy_img,gt)
    eval_dict['csv_noisy'] = csv_noisy
    eval_dict['csv_denoisy'] = csv_denoisy
    # ******************* SUMMER *******************
    summer_noisy = metrics.summer(noisy_img,gt)
    summer_denoisy =metrics.summer(denoisy_img,gt)
    eval_dict['summer_noisy'] = summer_noisy
    eval_dict['summer_denoisy'] = summer_denoisy

    return eval_dict

def list_of_dicts_to_dataframe(metr,save_pth):
    df = pd.DataFrame.from_dict({(i, j): metr[i][j] for i in metr.keys() for j in range(len(metr[i]))}, orient='index')
    lst = df.columns
    new_lst = ['Type of Noise','junk']
    for i in lst:
        new_lst.append(i)

    df.reset_index(inplace=True)
    df.columns = new_lst
    df.to_csv(save_pth,index=False)
    return df

## Denosing Algorithm - 1 --> Wavelet + NLMeans

In [12]:
h = 10  
tempWinSize = 7
searchWinSize = 21

def denoisy_img(lst):
    comb_denoisy_imgs = []
    for img in lst:
        # Apply NLMeans denoising to the current image
        denoised_img = combine_denoising_methods(img)
        comb_denoisy_imgs.append(denoised_img)  
    return comb_denoisy_imgs

denoised_imgs = {}
for name, imgs in noisy_imgs.items():
    denoised_imgs[name] = denoisy_img(imgs)


In [13]:
# df = []
metr = {}
for name, de_im in denoised_imgs.items():
    noi_im = noisy_imgs[name]
    lst = []
    for noi,denoi,gt in zip(noi_im,de_im,gt_imgs):
        lst.append(evaluation_metrics(denoi,noi,gt))
    metr[name] = lst

save_pth = r"C:\Users\ajayr\Documents\Masters\3rd Sem\Digital Image Processing\Project\Results\Denoising\Wavelet+NLMeans\CURE-TSD\cure-tsr_results.csv"
df = list_of_dicts_to_dataframe(metr,save_pth)


In [ ]:
# for noi,denoi,gt in zip(noisy_imgs,comb_denoisy_imgs,gt_imgs):
#     display_plots(gt,noi,denoi)

## Denosing Algorithm - 2 --> -Unet

In [14]:
model_path = 'Models\model_TSD.h5'
model = load_model(model_path)

[2023-11-30 15:18:31,661] DEBUG: Creating converter from 3 to 5
[2023-11-30 15:18:31,883] WARNING: From c:\Users\ajayr\miniconda3\envs\dip_project\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.

[2023-11-30 15:18:31,930] WARNING: From c:\Users\ajayr\miniconda3\envs\dip_project\lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [15]:
def normalize_image(img):
    # Check if range is zero
    if np.max(img) - np.min(img) == 0:
        # Handle this case (e.g., set all values to 0 or 1)
        img_normalized = img 
    else:
        # Normalize the image
        img_normalized = (img - np.min(img)) / (np.max(img) - np.min(img))
    
    return img_normalized
def autounet(lst):
    denoised_imgs =[]
    for img in lst:
        
        img = normalize_image(img)
        img = cv2.resize(img, (128, 128))
        img=np.reshape(img,(1,128,128,3))
        denoised_img = model.predict(img)
        denoised_img=np.reshape(denoised_img, (128, 128, 3))
        denoised_img = (denoised_img * 255).astype(np.uint8)

        # Apply NLMeans denoising to the current image
        # denoised_img = combine_denoising_methods(img)
        denoised_imgs.append(denoised_img)  
    return denoised_imgs
    
    

In [16]:
denoised_02_imgs = {}
for name, imgs in noisy_imgs.items():
    denoised_02_imgs[name] = autounet(imgs)

1/1 [==============================] - 0s 42ms/step


In [17]:
# df = []
metr = {}
for name, de_im in denoised_02_imgs.items():
    noi_im = noisy_imgs[name]
    lst = []
    for noi,denoi,gt in zip(noi_im,de_im,gt_imgs):
        noi=cv2.resize(noi,(128,128))
        gt=cv2.resize(gt,(128,128))
        lst.append(evaluation_metrics(denoi,noi,gt))
    metr[name] = lst

save_pth = r"C:\Users\ajayr\Documents\Masters\3rd Sem\Digital Image Processing\Project\Results\Denoising\Deep learning\CURE-TSD\cure-tsr_results.csv"
df = list_of_dicts_to_dataframe(metr,save_pth)


## Image Enhancement - Histogram matching

In [19]:
#gt_imgs,
def enhance_img(noisy_lst,gt_lst):
    comb_en_imgs = []
    for gt_img,noisy_img in (zip(gt_lst,noisy_lst)):
        # Apply NLMeans denoising to the current image
        en_img = match_histograms(noisy_img, gt_img, channel_axis=-1)
        comb_en_imgs.append(en_img)  
    return comb_en_imgs

enhan_imgs = {}
for name, imgs in noisy_imgs.items():
    enhan_imgs[name] = enhance_img(imgs,gt_imgs)

In [ ]:
# df = []
metr = {}
for name, de_im in enhan_imgs.items():
    noi_im = noisy_imgs[name]
    lst = []
    for noi,denoi,gt in zip(noi_im,de_im,gt_imgs):
        lst.append(evaluation_metrics(denoi,noi,gt))
    metr[name] = lst

save_pth = r"C:\Users\ajayr\Documents\Masters\3rd Sem\Digital Image Processing\Project\Results\Enhancement\CURE-TSD\cure-tsr_results.csv"
df = list_of_dicts_to_dataframe(metr,save_pth)
